In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl  # noqa: F401
    import dolfin  # noqa: F401
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl  # noqa: F401
        import dolfin  # noqa: F401

In [ ]:
try:
    import rbnics
except ImportError:
    !python3 -m pip install git+https://github.com/RBniCS/RBniCS.git
    import rbnics  # noqa: F401

In [ ]:
# Download data files
!mkdir -p data
![ -f data/thermal_block_facet_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/01_thermal_block/data/thermal_block_facet_region.xml -O data/thermal_block_facet_region.xml
![ -f data/thermal_block_physical_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/01_thermal_block/data/thermal_block_physical_region.xml -O data/thermal_block_physical_region.xml
![ -f data/thermal_block.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/01_thermal_block/data/thermal_block.xml -O data/thermal_block.xml

In [ ]:
from dolfin import *
from rbnics import *

In [ ]:
@SCM()
class ThermalBlock(EllipticCoerciveCompliantProblem):

    # Default initialization of members
    @generate_function_space_for_stability_factor
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        EllipticCoerciveCompliantProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        # Customize eigen solver parameters
        self._eigen_solver_parameters.update({
            "bounding_box_minimum": {
                "problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                "spectral_shift": 1.e-5, "linear_solver": "mumps"
            },
            "bounding_box_maximum": {
                "problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                "spectral_shift": 1.e5, "linear_solver": "mumps"
            },
            "stability_factor": {
                "problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                "spectral_shift": 1.e-5, "linear_solver": "mumps"
            }
        })

    # Return custom problem name
    def name(self):
        return "ThermalBlock"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_stability_factor
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = mu[0]
            theta_a1 = 1.
            return (theta_a0, theta_a1)
        elif term == "f":
            theta_f0 = mu[1]
            return (theta_f0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_stability_factor
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            u = self.u
            a0 = inner(grad(u), grad(v)) * dx(1)
            a1 = inner(grad(u), grad(v)) * dx(2)
            return (a0, a1)
        elif term == "f":
            ds = self.ds
            f0 = v * ds(1)
            return (f0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 3)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

In [ ]:
mesh = Mesh("data/thermal_block.xml")
subdomains = MeshFunction("size_t", mesh, "data/thermal_block_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/thermal_block_facet_region.xml")

In [ ]:
V = FunctionSpace(mesh, "Lagrange", 1)

In [ ]:
problem = ThermalBlock(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.1, 10.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)

In [ ]:
reduction_method = ReducedBasis(problem)
reduction_method.set_Nmax(4, SCM=4)
reduction_method.set_tolerance(1e-5, SCM=1e-5)

In [ ]:
reduction_method.initialize_training_set(10, SCM=10)
reduced_problem = reduction_method.offline()

In [ ]:
online_mu = (8.0, -1.0)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
plot(reduced_solution, reduced_problem=reduced_problem)

In [ ]:
reduction_method.initialize_testing_set(10, SCM=0)
reduction_method.error_analysis(SCM=1)

In [ ]:
%%bash

export LD_PRELOAD=""
ERROR_LIBRARIES=($(find /root/.cache/dijitso -name '*\.so' -exec \
    bash -c 'ldd $0 | grep libstdc++.so.6 1>/dev/null 2>/dev/null && echo $0' {} \;))
if [ ${#ERROR_LIBRARIES[@]} -eq 0 ]; then
    echo "No reference to libstdc++.so was found"
else
    for ERROR_LIBRARY in "${ERROR_LIBRARIES[@]}"; do
        echo "Error: library $ERROR_LIBRARY depends on libstdc++.so"
        ldd -v $ERROR_LIBRARY
    done
    false
fi